In [92]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn import preprocessing

warnings.filterwarnings(action='ignore')

# Part 5. 데이터 사전 처리

## 5-2. 누락 데이터 처리

#### 예제 5-1. 누락 데이터 확인

In [6]:
df = sns.load_dataset('titanic')
nan_deck = df['deck'].value_counts(dropna=False)
nan_deck

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

In [7]:
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [8]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [9]:
df.head().isnull().sum(axis=0)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64

#### 예제 5-2. 누락 데이터 제거

In [10]:
missing_df = df.isnull()

In [11]:
missing_df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
887,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
889,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [13]:
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts()
    
    try:
        print(col, ': ', missing_count[True])
    except:
        print(col, ': ', 0)

survived :  0
pclass :  0
sex :  0
age :  177
sibsp :  0
parch :  0
fare :  0
embarked :  2
class :  0
who :  0
adult_male :  0
deck :  688
embark_town :  2
alive :  0
alone :  0


In [16]:
# 결측치 값이 500 이상인 열을 아예 삭제
df_thresh = df.dropna(axis=1, thresh=500)
df_thresh.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [17]:
# age 열에 나이 데이터가 없는 행을 모두 삭제
df_age = df.dropna(subset=['age'], how='any', axis=0)
len(df_age)

714

#### 예제 5-3. 평균으로 누락 데이터 바꾸기

In [19]:
df['age'].head(10)

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64

In [20]:
mean_age = df['age'].mean(axis=0)
df['age'].fillna(mean_age, inplace=True)
df['age'].head(10)

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64

#### 예제 5-4. 가장 많이 나타나는 값으로 바꾸기

In [29]:
df = sns.load_dataset('titanic')

In [30]:
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object

In [31]:
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
most_freq

'Southampton'

In [32]:
df['embark_town'].fillna(most_freq, inplace=True)

In [33]:
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

#### 예제 5-5. 이웃하고 있는 값으로 바꾸기

In [34]:
# 바로 앞에 있는 행의 값으로 변경하기
df['embark_town'].fillna(method='ffill', inplace=True) 

In [36]:
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

## 5-2. 중복 데이터 처리

#### 예제 5-6. 중복 데이터 확인

In [39]:
df = pd.DataFrame({'c1' : ['a', 'a', 'b', 'a', 'b'],
                   'c2' : [1, 1, 1, 2, 2],
                   'c3' : [1, 1, 2, 2, 2]})
df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [40]:
df_dup = df.duplicated()
df_dup

0    False
1     True
2    False
3    False
4    False
dtype: bool

In [41]:
col_dup = df['c2'].duplicated()

In [42]:
col_dup

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool

#### 예제 5-7. 중복 데이터 제거

In [49]:
df = pd.DataFrame(dict(zip(['c' + str(i) for i in range(1, 4)], [['a', 'a', 'b', 'a', 'b'], [1, 1, 1, 2, 2], [1, 1, 2, 2, 2]])))

In [51]:
df2 = df.drop_duplicates()

In [52]:
df2

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [55]:
# 특정 열을 기준으로 중복 여부 판단
df3 = df.drop_duplicates(subset=['c2', 'c3'])
df3

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


## 5-3. 데이터 표준화

#### 예제 5-8. 단위 환산

In [67]:
df_car = pd.read_csv('samples/part5/auto-mpg.csv', header = None)
df_car.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']
df = df_car[:]

In [68]:
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite


In [69]:
mpg_to_kpl = 1.60934 / 3.78541
df['kpl'] = df['mpg'] * mpg_to_kpl
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.652571
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377143
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.652571


In [70]:
df['kpl'] = df['kpl'].round(2)
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65


#### 예제 5-9. 자료형 변환

In [73]:
df = df_car[:]
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
dtype: object

In [74]:
df['horsepower'].unique()

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', '?', '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [75]:
df['horsepower'].replace('?', np.nan, inplace = True) # ? -> np.nan
df.dropna(subset=['horsepower'], axis=0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

df['horsepower'].dtypes

dtype('float64')

In [76]:
df['origin'].unique()

array([1, 3, 2], dtype=int64)

In [77]:
df['origin'].replace({1:'USA', 2:'EU', 3:'JPN'}, inplace=True)
df['origin'].unique()

array(['USA', 'JPN', 'EU'], dtype=object)

In [81]:
df['origin'].dtypes

dtype('O')

In [82]:
df['origin'] = df['origin'].astype('category')
df['origin'].dtypes

CategoricalDtype(categories=['EU', 'JPN', 'USA'], ordered=False)

In [83]:
df['origin'] = df['origin'].astype('str')
df['origin'].dtypes

dtype('O')

## 5-4. 범주형(카테고리) 데이터 처리

#### 예제 5-10. 데이터 구간 분할

In [84]:
df = df_car[:]

In [85]:
df['horsepower'].replace('?', np.nan, inplace = True) # ? -> np.nan
df.dropna(subset=['horsepower'], axis=0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

In [88]:
# 경계값 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
bin_dividers

array([ 46.        , 107.33333333, 168.66666667, 230.        ])

In [89]:
bin_names = ['저출력', '보통출력', '고출력']
df['hp_bin'] = pd.cut(x=df['horsepower'], bins=bin_dividers, labels=bin_names, include_lowest = True)
df[['horsepower', 'hp_bin']].head(15)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


#### 예제 5-11. 더미 변수

In [91]:
horsepower_dummies = pd.get_dummies(df['hp_bin'])
horsepower_dummies.head(15)

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,1


#### 예제 5-12. 원핫인코딩

In [94]:
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
onehot_labeled

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2])

In [95]:
type(onehot_labeled)

numpy.ndarray

In [97]:
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
onehot_reshaped

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [2]])

In [98]:
type(onehot_reshaped)

numpy.ndarray

In [102]:
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)

  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0


In [103]:
type(onehot_fitted)

scipy.sparse.csr.csr_matrix

## 5-5. 정규화

#### 예제 5-13. 정규화

In [104]:
df = df_car[:]

In [105]:
df['horsepower'].replace('?', np.nan, inplace = True) # ? -> np.nan
df.dropna(subset=['horsepower'], axis=0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

In [106]:
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


In [107]:
df.horsepower = df.horsepower/abs(df.horsepower.max())

In [109]:
df.horsepower.head()

0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64

In [111]:
df.horsepower.describe()

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64

#### 예제 5-14. 정규화

In [112]:
df = df_car[:]
df['horsepower'].replace('?', np.nan, inplace = True) # ? -> np.nan
df.dropna(subset=['horsepower'], axis=0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


In [113]:
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x/min_max
df.horsepower.head()

0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64

In [114]:
df.horsepower.describe()

count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64

## 5-6. 시계열 데이터 

#### 예제 5-15. 문자열을 Timestamp로 변환

In [117]:
df_stock = pd.read_csv('samples/part5/stock-data.csv')
df = df_stock[:]

In [118]:
df.head()

,Date,Close,Start,High,Low,Volume
0,2018-07-02,10100,10850,10900,10000,137977
1,2018-06-29,10700,10550,10900,9990,170253
2,2018-06-28,10400,10900,10950,10150,155769
3,2018-06-27,10900,10800,11050,10500,133548
4,2018-06-26,10800,10900,11000,10700,63039


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


In [120]:
df['new_Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,Start,High,Low,Volume,new_Date
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB


In [122]:
type(df['new_Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [124]:
df.set_index('new_Date', inplace=True)
df.drop('Date', axis=1, inplace=True)

In [125]:
df.head()

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes


#### 예제 5-16. Timestamp를 Period로 변환

In [128]:
dates = ['2019-01-01', '2020-03-01', '2021-06-01']
ts_dates = pd.to_datetime(dates)
ts_dates

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)

In [129]:
pr_day = ts_dates.to_period(freq='D')

In [130]:
pr_day

PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')

In [131]:
pr_month = ts_dates.to_period(freq='M')

In [132]:
pr_month

PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')

In [133]:
pr_year = ts_dates.to_period(freq='A')

In [134]:
pr_year

PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')

#### 예제 5-17. Timestamp 배열 만들기

In [137]:
ts_ms = pd.date_range(start='2019-01-01', end=None, periods=6, freq='MS', tz='Asia/Seoul')
ts_ms

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')